# Extractor tutorial for Pic du Midi Data

The goal of this notebook is to demonstrate the capabilities of the Spectractor pipeline (still under construction). This pipeline is designed to extract a spectrum from an image given the coordinate (in pixels) of the object.

## Import the pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
import os
import pandas as pd
import re

In [3]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print('workbookDir: ' + workbookDir)
#os.chdir(workbookDir)  # If you changed the current working dir, this will take you back to the workbook dir.

workbookDir: /Users/dagoret/MacOSX/GitHub/LSST/Spectractor/notebooks


In [4]:
import sys
sys.path.append(workbookDir)
sys.path.append(os.path.dirname(workbookDir))

In [5]:
from spectractor import parameters
from spectractor.extractor.extractor import Spectractor
from spectractor.logbook import LogBook

if __name__ == "__main__":
    from argparse import ArgumentParser

    parser = ArgumentParser()
    parser.add_argument(dest="input", metavar='path', default=["tests/data/reduc_20170605_028.fits"],
                        help="Input fits file name. It can be a list separated by spaces, or it can use * as wildcard.",
                        nargs='*')
    parser.add_argument("-d", "--debug", dest="debug", action="store_true",
                        help="Enter debug mode (more verbose and plots).", default=False)
    parser.add_argument("-v", "--verbose", dest="verbose", action="store_true",
                        help="Enter verbose (print more stuff).", default=False)
    parser.add_argument("-o", "--output_directory", dest="output_directory", default="outputs/",
                        help="Write results in given output directory (default: ./outputs/).")
    parser.add_argument("-l", "--logbook", dest="logbook", default="ctiofulllogbook_jun2017_v5.csv",
                        help="CSV logbook file. (default: ctiofulllogbook_jun2017_v5.csv).")
    parser.add_argument("-c", "--config", dest="config", default="config/ctio.ini",
                        help="INI config file. (default: config.ctio.ini).")
    args = parser.parse_args()


# Configuration

## Flags

In [6]:
parameters.VERBOSE = True
parameters.DEBUG = True

## Date

In [7]:
thedate="20190214"
#thedate="20190215"

## Logbook

In [8]:
logbookfilename="../simple_logbook_PicDuMidi_"+thedate+".csv"
#logbookfilename="../ctiofulllogbook_jun2017_v5.csv" 

In [9]:
# good for my logbook
df=pd.read_csv(logbookfilename,sep=",", decimal=".", encoding='latin-1', header='infer')

In [10]:
df.file[0]

'T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit'

## Input filename

In [11]:
if thedate=="20190215":
    INPUTDIR="/Users/dagoret/DATA/PicDuMidiFev2019/prod_20190215"
else:
    INPUTDIR="/Users/dagoret/DATA/PicDuMidiFev2019/prod_20190214"

In [12]:
if thedate=="20190215":
    file_name="T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit"
else:
    file_name="T1M_20190214_234122_495_HD116405-Ronchi_Filtre_None_bin1x1.1_red.fit"    

## Output directory

In [13]:
output_directory="output"

## Config

In [14]:
config="../config/picdumidi.ini"

# Run Logbook

In [25]:
df=pd.read_csv(logbookfilename)

In [26]:
df.head()

,Unnamed: 0,object,disperser,filt,evnum,flat,airmass,date,exposure,gain,RA,DEC,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,file
0,0,HD116405,Holo150PDM19,None,1,FLAT_Filtre_None_bin1x1,1.545876,2019-02-15T22:55:50.730,30.0,4.8,13.379201,44.714975,133.208557,220.874719,50.0,50.0,No,T1M_20190215_225550_730_HD116405_Filtre_None_b...
1,1,HD116405,Holo150PDM19,None,2,FLAT_Filtre_None_bin1x1,1.542852,2019-02-15T22:56:26.007,30.0,4.8,13.379201,44.714975,133.400674,217.612274,50.0,50.0,No,T1M_20190215_225626_007_HD116405_Filtre_None_b...
2,2,HD116405,Holo150PDM19,None,3,FLAT_Filtre_None_bin1x1,1.539842,2019-02-15T22:57:01.284,30.0,4.8,13.379201,44.714975,133.268162,217.954960,50.0,50.0,No,T1M_20190215_225701_284_HD116405_Filtre_None_b...
3,3,HD116405,Holo150PDM19,None,4,FLAT_Filtre_None_bin1x1,1.536846,2019-02-15T22:57:36.562,30.0,4.8,13.379201,44.714975,183.596980,174.983184,50.0,50.0,No,T1M_20190215_225736_562_HD116405_Filtre_None_b...
4,4,HD116405,Holo150PDM19,None,5,FLAT_Filtre_None_bin1x1,1.533865,2019-02-15T22:58:11.838,30.0,4.8,13.379201,44.714975,180.257505,174.547611,50.0,50.0,No,T1M_20190215_225811_838_HD116405_Filtre_None_b...


In [28]:
df.iloc[0]["file"]

'T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit'

In [15]:
logbook = LogBook(logbookfilename)

In [16]:
tag_file=file_name

In [17]:
disperser_label, target, xpos, ypos = logbook.search_for_image(tag_file)

In [18]:
disperser_label

'Holo150PDM19'

In [19]:
target

'HD116405'

In [20]:
xpos,ypos

(133, 220)

In [21]:
fullfilename=os.path.join(INPUTDIR,file_name)

In [22]:
fullfilename

'/Users/dagoret/DATA/PicDuMidiFev2019/prod_20190215/T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit'

In [23]:
#assert False,"STOP" 

# Run Spectractor

In [24]:
Spectractor(fullfilename,output_directory, [xpos, ypos], target, disperser_label, config,logbook=logbookfilename)

2019-03-26 12:15:17  spectractor.extractor.extractor Spectractor          INFO   
	Start SPECTRACTOR
2019-03-26 12:15:17  Image      load_PDM_image       INFO   
	Loading Pic du Midi image /Users/dagoret/DATA/PicDuMidiFev2019/prod_20190215/T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit...
2019-03-26 12:15:17  Image      load_PDM_image       INFO   
	Image loaded
2019-03-26 12:15:17  Image      load_LogBook         INFO   
	Load Logbook  ../simple_logbook_PicDuMidi_20190215.csv for /Users/dagoret/DATA/PicDuMidiFev2019/prod_20190215/T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit...
2019-03-26 12:15:17  Image      load_LogBook         INFO   
	Load Logbook  : date identified  20190215 for /Users/dagoret/DATA/PicDuMidiFev2019/prod_20190215/T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit...


Section: paths
x holo_dir: extractor/dispersers/	 => parameters.HOLO_DIR: /Users/dagoret/MacOSX/GitHub/LSST/Spectractor/spectractor/extractor/dispersers/	 <class 'str'>
x throughput_dir: simulation/CTIOThroughput/	 => parameters.THROUGHPUT_DIR: /Users/dagoret/MacOSX/GitHub/LSST/Spectractor/spectractor/simulation/CTIOThroughput/	 <class 'str'>
Section: instrument
x obs_name: PICDUMIDI	 => parameters.OBS_NAME: PICDUMIDI	 <class 'str'>
x obs_altitude: 2.8905	 => parameters.OBS_ALTITUDE: 2.8905	 <class 'float'>
x obs_longitude: 0 08 34.00	 => parameters.OBS_LONGITUDE: 0 08 34.00	 <class 'str'>
x obs_latitude: 42 56 11.00	 => parameters.OBS_LATITUDE: 42 56 11.00	 <class 'str'>
x obs_diameter: 1.0	 => parameters.OBS_DIAMETER: 1.0 m	 <class 'astropy.units.quantity.Quantity'>
x obs_epoch: J2000.0	 => parameters.OBS_EPOCH: J2000.0	 <class 'str'>
x obs_transmission_systematics: 0.005	 => parameters.OBS_TRANSMISSION_SYSTEMATICS: 0.005	 <class 'float'>
x obs_object_type: STAR	 => parameters.OBS_OB

IndexError: list index out of range